<a href="https://colab.research.google.com/github/aghadavood/NLP/blob/main/PesrianQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%%capture
!pip install transformers

In [16]:
%%capture
!pip install datasets

In [18]:
import transformers
model_checkpoint = "HooshvareLab/bert-fa-zwnj-base"
batch_size = 1

In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
from datasets import load_dataset

datasets = load_dataset("SajjadAyoubi/persian_qa")

In [22]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [23]:
from datasets import ClassLabel ,Sequence
import random
import pandas as pd
from IPython.display import display , HTML

def show_random_elements (dataset , num_examples = 10):
  assert num_examples <= len(dataset) , "can't pick more elements than there are in the dataset."
  picks = []
  for _ in range (num_examples):
    pick = random.randint (0, len(dataset) - 1)
    while pick in picks :
      pick = random.randint(0 , len(dataset) - 1)
    picks.append(pick)

  df = pd.DataFrame(dataset[picks])
  for column , typ in dataset.features.items():
    if isinstance(typ , ClassLabel):
      df[column] = df[column].transform (lambda i: typ.name[i])
    elif isinstance(typ , Sequence) and isinstance(typ.feature , ClassLabel):
        df[column] = df [column].transform(lambda x: [typ.feature.names[i] for i in x])
  display (HTML(df.to_html()))



In [24]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,6949,تهیه‌کننده فیلم,تهیه‌کننده فیلم کسی است که شرایط را برای ساخت فیلم فراهم می‌کند. راه‌اندازی، هماهنگ‌سازی، رسیدگی و کنترل کارهایی از جمله انتخاب قصه و فیلم‌نامه، برنامه‌ریزی تولید، جمع‌آوری پول مورد نیاز، به‌کارگیری افراد کلیدی و این‌گونه کارها بر عهدهٔ تهیه‌کننده است. تهیه‌کننده در تمام مراحل فرایند فیلم‌سازی و برنامه‌سازی از توسعه تا تکمیل پروژه نقش جدی دارد و همه عوامل فیلم اعم از کارگردان و بازیگران تا سایر عوامل به‌طور مستقیم یا غیر مستقیم زیرنظر تهیه‌کننده فیلم فعالیت می‌کنند. در فرایند برنامه‌سازی رادیویی و تلویزیونی، کلیدی‌ترین نقش را تهیه‌کننده اجرا می‌کند همچنین در ساخت انیمیشن نیز مهم‌ترین نقش بر عهده تهیه‌کننده است، بر خلاف فیلم‌سازی سینمایی که در آن وظیفهٔ مهم تشکیل فرم و ساختار فیلم اساساً متکی به دانش و خلاقیت کارگردان است. در نیمهٔ نخست قرن بیستم، تهیه‌کنندگان می‌بایست ادارهٔ کنترل هنری فیلم را هم انجام می‌دادند. در ایالات متحده با کنار گذاشتن دستگاه استودیویی هالیوود در دههٔ ۱۹۵۰، کنترل هنری کار بر عهدهٔ دستیاران کارگردان گذاشته شد.,تهیه کننده چیکار نمی کنه؟,"{'text': [], 'answer_start': []}"
1,2114,گردو,"درخت گردو از درختان بسیار با ارزش و از پهن برگان است. مغز گردو دارای چند اسید آلی، مقدر کمی اسانس، اگزالات آهک و ویتامین‌های A ,B ,C ,D ,E می‌باشد. عصاره برگ درخت گردو خاصیت میکروب کشی و باکتری کشی دارد. درخت گردو بیشتر به روش پیوندزدن (پیوند اسکنه‌ای، پیوند شکمی با T معکوس به دلیل وجود شیرآبه) تکثیر می‌شود. گردو درختی بزرگ و گسترده است؛ و بیشتر از هر درخت دیگر به نور نیاز دارد. گردو در سن بلوغ میوه می‌دهد؛ و در قسمت‌هایی از درخت که نور بیشتری می‌تابد از باردهی بهتری برخوردار است؛ و در قسمت‌های داخلی که نور به آن نمی‌تابد گردوها ریز و کوچک می‌شوند. بعد از آنکه گردو کاشته شد و شروع به رشد کرد. (به اندازه ای رسید که بتوان آن را جابجا کرد) نهال گردو را با ریشه از خاک درآورده و در چاله‌های عمیق حفر شده در زمینی که خاکش از نظر کود دهی آماده شده‌است کاشته می‌شود. و با خاک اطرافش را می‌پوشانند. تا برای آبیاری درخت مشکلی ایجاد نشود . بهترین فصل کاشت درخت گردو اوایل بهار است. در کاشت درخت گردو باید دقت کرد که درخت ۲۰–۳۰ سانتیمتر بالاتر از کف جوی کاشته شود. تا در آینده به بیماری آب سیاه مبتلا نشود. در احداث باغ‌های جدید باید سعی کرد حتماً از نهال پیوندی استفاده شود.",روش کاشت گردو را بنویسید؟,"{'text': ['بعد از آنکه گردو کاشته شد و شروع به رشد کرد. (به اندازه ای رسید که بتوان آن را جابجا کرد) نهال گردو را با ریشه از خاک درآورده و در چاله‌های عمیق حفر شده در زمینی که خاکش از نظر کود دهی آماده شده‌است کاشته می‌شود.'], 'answer_start': [555]}"
2,2519,نهنگ قاتل,نهنگ قاتل که به نام اُرکا(Orcinus orca) نیز شناخته می‌شود، پستانداری دریایی است که به خانواده دلفین‌های اقیانوسی تعلّق دارد و بزرگ‌ترین جاندار در میان دلفین‌سانان است. نهنگ‌های قاتل در اقیانوس‌های سراسر جهان، از دریاهای قطبی گرفته تا دریاهای گرمسیری، یافت می‌شوند. آن‌ها طعمه‌های گوناگونی را به‌عنوان غذا اختیار می‌کنند، با این حال بعضی گروه‌ها تنها به دنبال یک طعمه می‌روند. بعضی تنها از ماهی‌ها تغذیه می‌کنند و برخی دیگر به گله‌های پستانداران دریایی، مانند فک، شیر دریایی، گراز دریایی و حتی نهنگ‌ها نیز حمله می‌کنند. نهنگ‌های قاتل در بالای هرم غذایی هستند، دشمنان طبیعی ندارند و گاه کوسه‌های بزرگ را نیز طعمه خود می‌کنند. نهنگ‌های قاتل بسیار اجتماعی‌اند؛ بعضی دسته‌های آن‌ها دارای خانواده‌های مادر نهاد هستند که پایدارترین خانواده‌های این چنینی در میان همه جانوران را تشکیل می‌دهند.روش‌های شکار پیچیده و صداهای تولیدیشان، که اغلب منحصر به یک دسته‌اند و نسل به نسل منتقل می‌شوند، به عنوان گونه‌ای از فرهنگ شناخته می‌شوند.,دشمنان نهنگ قاتل از چه چیزی تغذیه میکنند؟,"{'text': [], 'answer_start': []}"
3,3537,مه‌بانگ,نظریه مِه‌بانگ یا بیگ بنگ تولد زمان و مدل کیهان‌شناسی پذیرفته‌شده جهان، از کهن‌ترین دوران شناخته‌شده و تکامل آن در اندازه و مقیاس بزرگ است. این نظریه بیان می‌کند که گیتی از یک وضعیت بسیار چگال (متراکم) نخستین آغاز شده و در گذر زمان انبساط یافته‌است. این نظریه طیف گسترده‌ای از پدیده‌های مشاهده‌شده را به خوبی توضیح می‌دهد. از جمله این پدیده‌ها می‌توان به فراوانی عناصر سبک اولیه، تابش زمینه کیهانی، ساختار بزرگ مقیاس و قانون هابل اشاره نمود. اگر در زمان به عقب برگردیم، به نقطه‌ای در گذشته می‌رسیم که در آن قوانین فیزیکی شناخته‌شده کارای

In [25]:
show_random_elements(datasets["validation"])

,id,title,context,question,answers
0,9532,والیبال,والیبال یک ورزش گروهی و ششمین ورزش پر طرفدار و گسترده‌ترین ورزش در بعضی کشورهای جهان است که در آن بازیکنان در دو تیم شش نفره، در دو سوی توری قرار می‌گیرند و تلاش می‌کنند تا طبق قوانین بازی، توپ را از روی تور در زمین تیم مقابل فرود آورند. طول زمین والیبال ۱۸ متر و عرض آن ۹ است. هرتیم حداقل باید ۳ ست (یا دست) از ۵ پنج ست بازی را ببرد، تا بتواند پیروز مسابقه شود. والیبال در سال ۱۸۹۵ در ایالات متحده آمریکا و توسط ویلیام مورگان ابداع شد. سپس به‌سرعت فراگیر شد به‌طوری‌که در بازی‌های المپیک تابستانی ۱۹۶۰ برای نخستین‌بار در برنامه بازی‌های المپیک قرار گرفت و امروزه تقریباً در تمام دنیا بازی می‌شود. این رشته تقریباً برای تمام گروه‌های سنی مناسب است و وسایل بازی شامل توپ و تور است. امروزه، بیش از ۸۰۰ میلیون بازیکن والیبال در سراسر جهان وجود دارند، که ۴۶ میلیون نفر از آن‌ها از ایالات متحده آمریکا هستند. بر اساس اطلاعات کمیته المپیک، والیبال بالاتر از فوتبال، تنیس و دیگر ورزش‌ها محبوب‌ترین ورزش بازی‌های المپیک است.,تیم ها تو والیبال چند ست ببرن برنده بازی می شن؟,"{'text': ['۳ ست (یا دست) از ۵ پنج ست', '۳ ست (یا دست)'], 'answer_start': [295, 295]}"
1,9559,فرودگاه بین‌المللی مهرآباد,فرودگاه بین‌المللی مهرآباد نام یکی از فرودگاه‌های شهر تهران، پایتخت ایران است که در سال ۱۳۱۷ ساخته شده‌است. این فرودگاه در بخش غربی تهران واقع شده و نام خود را از روستای مهرآباد جنوبی ده بالا که پیشتر در آن منطقه قرار داشت گرفته‌است. این فرودگاه مجهز به سامانه‌های آی ال اس و وی او آر است. فرودگاه مهرآباد پس از فرودگاه مسجد سلیمان، فرودگاه بندر لنگه، قلعه مرغی و فرودگاه خرم‌آباد، پنجمین فرودگاه ایران می‌باشد. در سال ۲۰۱۵ میلادی ۱۳۲٬۲۴۵ نشست و برخاست هواپیما در این فرودگاه انجام شد و ۱۱۱٬۱۰۷٬۸۲۲ کیلوگرم بار و ۱۶٬۳۲۷٬۳۵۹ نفر مسافر از طریق آن جابجا شدند. نام اصلی روستای مهرآباد «حسین‌آباد» و در تملک حاجی میرزا آقاسی صدراعظم محمد شاه قاجار بود. حاجی میرزا آقاسی، حسین‌آباد را به ناصرالدین شاه هدیه کرد و بعد از مدتی جزو مهریه عصمت‌الدوله (دختر ناصرالدین‌شاه) و همسر دوستمحمدخان معیرالممالک، درآمد و نام مهرآباد را بر آن گذاشتند. اما بعدها به مناسبت خرج‌های بی‌رویه دوستمحمدخان، مهرآباد به گرو رفت و از تملک خاندان معیرالممالک خارج شد و دادخواهی دوستعلی معیری پسر دوست‌محمدخان نیز نزد رضاشاه بی‌فایده بود.,بهترین شرکت هواپیمایی دنیا کدوم شرکته؟,"{'text': [], 'answer_start': []}"
2,10316,یووال نوح هراری,یووال نوح هراری (به عبری: יובל נח הררי) (زاده ۲۴ فوریهٔ ۱۹۷۶) روشن‌فکر، تاریخ‌دان و نویسنده اسرائیلی و استاد تاریخ دانشگاه عبری اورشلیم است که دو کتاب پرفروش بین‌المللی انسان خردمند: تاریخ مختصر بشر و انسان خداگونه: تاریخ مختصر آینده را نوشته‌است. کسانی چون بیل گیتس و باراک اوباما نوشته‌های یووال هراری را ستوده‌اند. او در سال ۲۰۱۸ کتاب بیست‌ویک درس برای سده بیست‌ویکم را منتشر کرد.هراری در ۲۴ فوریه ۱۹۷۶ در شهر کریات آتا در شمال اسرائیل در یک خانواده سکولار یهودی با ریشه‌هایی از اروپای شرقی و لبنانی زاده شد. او دارای مدرک دکترای تاریخ از دانشگاه آکسفورد می‌باشد و هم‌اکنون در دانشگاه عبری اورشلیم تدریس می‌کند. یووال نوح هراری آشکارا همجنسگراست. او با همسرش، ایتزیک یهو که مدیر برنامه‌هایش نیز هست و او را «اینترنت من از همه‌چیز» می‌خواند، در سال ۲۰۰۲ آشنا شد. آن‌ها در یک مراسم سنتی در شهر تورنتو در کانادا ازدواج کردند و هم‌اکنون در حومه تل‌آویو زندگی می‌کنند. او از ژانویه ۲۰۱۹، تلفن هوشمند ندارد. نوح هراری یک گیاهخوار نیز هست.,هراری در کجا به دنیا امده است؟,"{'text': ['در شهر کریات آتا در شمال اسرائیل', 'شهر کریات'], 'answer_start': [406, 409]}"
3,10344,کریم بنزما,کریم مصطفی بنزما (به فرانسوی: Karim Mostafa Benzema) (زادهٔ ۱۹ دسامبر ۱۹۸۷ در لیون) بازیکن فوتبال اهل فرانسه است که برای باشگاه فوتبال رئال مادرید در پست مهاجم بازی می‌کند. بنزما در حال بازی برای فرانسه در یورو ۲۰۱۲ بنزما فوتبال حرفه‌ای خود را از باشگاه المپیک لیون آغاز کرد و در سال ۲۰۰۴ میلادی وارد ترکیب تیم بزرگسالان این باشگاه شد و پس از ۵ سال بازی برای این تیم، در فصل ۲۰۰۹–۱۰ به باشگاه رئال مادرید پیوست. مبلغ انتقال قرارداد او از لیون به رئال ۳۵ میلیون یورو اعلام شد که با توجه به نحوه بازی او و عملکرد مثبتش این قرارداد به تا سقف ۴۱ میلیون یورو نیز افزایش می‌یافت. قرارداد او در سالی وارد باشگاه رئال مادرید شد که ستارگانی همچون کریستیانو رونالدو و کاکا نیز در آن سال با ای

In [26]:
max_length = 384
doc_stride = 128

In [27]:
pad_on_right = tokenizer.padding_side == "right"

In [28]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:

                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples



In [29]:
features = prepare_train_features(datasets['train'][:5])

In [30]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)


In [31]:
from transformers import AutoModelForQuestionAnswering , TrainingArguments , Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [33]:
torch.cuda.is_available()

True

In [34]:
%%capture
!pip install accelerate -U
!pip install transformers -U

In [1]:
!pip install torch torchvision -U


In [2]:
!pip uninstall accelerate transformers -y
!pip install accelerate transformers -U


Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Successfully uninstalled accelerate-0.28.0
Found existing installation: transformers 4.39.2
Uninstalling transformers-4.39.2:
  Successfully uninstalled transformers-4.39.2
  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Using cached transformers-4.39.2-py3-none-any.whl (8.8 MB)


In [46]:
from transformers import Trainer, default_data_collator
from transformers import FlaxAutoModelForQuestionAnswering , AutoTokenizer

# define model name and tokenizer name
model_name = "parsbert-finetuned-persianQA"


#load model and tokenizer
#model = AutoModelForQuestionAnswering.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(toke)
args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    save_steps=5000
)

data_collator = default_data_collator

trainer = Trainer(
    model.to(device),
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.882000,2.424150
2,2.087100,2.244498
3,1.430800,2.358090


TrainOutput(global_step=5523, training_loss=2.270387092146907, metrics={'train_runtime': 2229.7524, 'train_samples_per_second': 12.385, 'train_steps_per_second': 2.477, 'total_flos': 5411782452948480.0, 'train_loss': 2.270387092146907, 'epoch': 3.0})

In [49]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [50]:
output.start_logits.shape , output.end_logits.shape

(torch.Size([5, 384]), torch.Size([5, 384]))

In [52]:
output.start_logits.argmax(dim = -1) , output.end_logits.argmax (dim = -1)

(tensor([ 19,  62, 128, 157,  21], device='cuda:0'),
 tensor([ 19,  62, 137, 159,  24], device='cuda:0'))

In [53]:
n_best_size = 20

In [54]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

valid_answers = []
for start_index in start_indexes:
  for end_index in end_indexes:
    if start_index <= end_index:
      valid_answers.append(
          {
              "score" : start_logits[start_index] + end_logits[end_index] ,
              "text" : ""
          }
      )

In [55]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [56]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

In [57]:
raw_predictions = trainer.predict(validation_features)


In [58]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
max_answer_length = 30


In [59]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
context = datasets["validation"][0]["context"]
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index:
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 14.389643, 'text': 'مادرید'},
 {'score': 12.133005, 'text': 'در مادرید'},
 {'score': 11.992829,
  'text': 'رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 11.970998,
  'text': 'باشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 11.476729,
  'text': 'مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 10.117722, 'text': 'رئال مادرید'},
 {'score': 10.09589, 'text': 'باشگاه فوتبال رئال مادرید'},
 {'score': 9.601622, 'text': 'مادرید'},
 {'score': 9.188358, 'text': 'یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 8.788071, 'text': 'مادرید، پایتخت اسپانیا قرار دارد.'},
 {'score': 8.344689, 'text': 'مادرید، پایتخت اسپانیا قرار دارد'},
 {'score': 7.7260566, 'text': 'باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 7.324646, 'text': 'مادرید،'},
 {'score': 6.5314345, 'text': 'در مادرید، پایتخت اسپانیا قرار دارد.'},
 {'score': 6.3968377, 'text': 'مادرید، پایتخت اسپانیا'},
 {'score': 6.391

In [60]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [61]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()


    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []

        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [62]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)


Post-processing 930 example predictions split into 930 features.


  0%|          | 0/930 [00:00<?, ?it/s]

In [64]:
from datasets import load_metric
metric = load_metric("squad_v2")
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

<ipython-input-64-b5c491321e97>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [65]:
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 40.75268817204301,
 'f1': 56.707707774863785,
 'total': 930,
 'HasAns_exact': 30.568356374807987,
 'HasAns_f1': 53.36124152169485,
 'HasAns_total': 651,
 'NoAns_exact': 64.51612903225806,
 'NoAns_f1': 64.51612903225806,
 'NoAns_total': 279,
 'best_exact': 40.96774193548387,
 'best_exact_thresh': 0.0,
 'best_f1': 56.922761538304655,
 'best_f1_thresh': 0.0}